In [28]:

%load_ext autoreload

%autoreload 2

from lstar_extraction.LSTM import LSTMNetwork
# from GRU import GRUNetwork
from lstar_extraction.RNNClassifier import RNNClassifier
from lstar_extraction.Training_Functions import mixed_curriculum_train,make_train_set_for_target,make_test_set
from lstar_extraction.Tomita_Grammars import tomita_1, tomita_2, tomita_3, tomita_4, tomita_5, tomita_6, tomita_7, tomita_8, tomita_9







The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
# make training set
target = tomita_9

alphabet = "lrab"

# alphabet = "abcd"
train_set = make_train_set_for_target(target,alphabet)

# define rnn
rnn = RNNClassifier(alphabet,num_layers=1,hidden_dim=10,RNNClass = LSTMNetwork)


print(train_set)


made train set of size: 395 , of which positive examples: 30
{'': True, 'r': False, 'R': False, 'l': False, 'L': False, 'LR': True, 'lr': True, 'Lr': False, 'rl': False, 'll': False, 'lL': False, 'lR': False, 'rL': False, 'LL': False, 'RL': False, 'Rr': False, 'Ll': False, 'rR': False, 'rr': False, 'Rl': False, 'RR': False, 'Rrr': False, 'lrL': False, 'RLr': False, 'LRL': False, 'rLr': False, 'llr': False, 'RlR': False, 'LrR': False, 'RrL': False, 'rRr': False, 'Lrl': False, 'llL': False, 'lRR': False, 'lLL': False, 'LlR': False, 'RLl': False, 'Lll': False, 'Rlr': False, 'lrR': False, 'rrr': False, 'lLRr': True, 'lrlr': True, 'LRLR': True, 'LRlr': True, 'LLRR': True, 'llrr': True, 'LlrR': True, 'lrLR': True, 'RLRL': False, 'rRrr': False, 'LRLr': False, 'rrlR': False, 'lLlL': False, 'lrRr': False, 'lrRR': False, 'LllR': False, 'rlrr': False, 'RRlR': False, 'RLLl': False, 'rLRl': False, 'rLrR': False, 'RrLr': False, 'rRLR': False, 'LrLR': False, 'RLLr': False, 'LrlR': False, 'lRrr': Fals

In [60]:
# train the model
mixed_curriculum_train(rnn,train_set,stop_threshold = 0.0005)
rnn.renew()  


current average loss is:  0.19445215473572414
current average loss is:  0.13974708408117295
current average loss is:  0.12972278172771137
current average loss is:  0.11243788803617158
current average loss is:  0.085654362787803
current average loss is:  0.06849878045577037
current average loss is:  0.06289611043809336
current average loss is:  0.05947892394246934
current average loss is:  0.05629581016830251
current average loss is:  0.05275359817698032
current average loss is:  0.06089839873214562
current average loss is:  0.044926479781667396
current average loss is:  0.0389688050126036
current average loss is:  0.04239143536581347
current average loss is:  0.03111137185245752
current average loss is:  0.028144454956054686
current average loss is:  0.0264485274689107
current average loss is:  0.02482417867153506
current average loss is:  0.023392906671837917
current average loss is:  0.022124910958205596
current average loss is:  0.02933416626416146
current average loss is:  0.018665

In [61]:
 # generate_test_set


test_set=make_test_set(alphabet)

from test_xRNN import Traces



traces=Traces(rnn, alphabet)
traces.label_from_network(test_set)
traces.write_in_file()

# starting examples
# all_words = sorted(list(train_set.keys()),key=lambda x:len(x))
# pos = next((w for w in all_words if rnn.classify_word(w)==True),None)
# neg = next((w for w in all_words if rnn.classify_word(w)==False),None)
# starting_examples = [w for w in [pos,neg] if not None == w]



made test set of size: 175


In [62]:


# rnn.renew() 

# extract DFA from RNN
from lstar_extraction.Extraction import extract


dfa, ltl = extract(rnn,alphabet,traces, time_limit = 50,initial_split_depth = 10,starting_examples=None)

# print(dfa)
print("\n\nfinal LTL formula:", ltl)
# dfa.draw_nicely(maximum=30) #max size willing to draw



provided counterexamples are: None



start formula depth: 1
Illegal character 'L' in the input formula


ValueError: Syntax error in input! LexToken(RPAR,')',1,4)